# **Entrez: Sistema de retorno de bases de datos (NCBI)**

## **Acceso con Biopython**

## Simón Villanueva

Entrez, es un sistema de retorno de datos que permite a los usuarios acceder a las [bases de datos del NCBI](https://www.ncbi.nlm.nih.gov/search/?expt_dict=b&utm_expid=.DBAxP8EQR36MD9GU0WtSdg.2&utm_referrer=) como PubMed, GenBank, GEO, etc.

### **Normas de Entrez**

Imaginense que son un base de datos que compila básicamente todo elconocimiento de las ciencias de la vida y que le ofreces acceso público a nivel mundial. ¿Cuantas solicitudes de datos creen que puedan generarse en un día? ¿En una hora? ¿En un minuto? ¿En un segundo?

Claramente, si ésto no se regula, hay que ir buscando otro "computador" (Y este sí que es _Grande_ $\rightarrow$ \$\$\$)

Entonces, para acceder a los datos hay que seguir las normas establecidas por el NCBI:

- Si se va a hacer una serie de más de 100 solicitudes, hacerla en fines de semana o en horas no-pico de USA
- Usar este link [https://www.ncbi.nlm.nih.gov/books/NBK25501/](https://www.ncbi.nlm.nih.gov/books/NBK25501/) para hacer las solicitudes $^*$. 
- No hacer más de 1 solicitud cada 3 segundos $^*$
- Identificarse: proveer un correo en caso de que el NCBI te quiera contactar. No usar correos inexistentes, es preferible no identificarse.
- Para incrementar la tasa de solicitudes, utilizar una API-key (hasta 10 solicitudes por segundo).

$^*$Biopython hace esto por nosotros

En caso de no seguir los lineamientos, el NCBI puede bloquearte el acceso a sus bases de datos.

### **EInfo: Información de las bases de datos**

Con **Einfo** podemos obtener información acerca de las bases de datos que tiene el NCBI. ¿Cómo vamos a utilizar la información del NCBI si no sabemos qué hay?

In [1]:
from Bio import Entrez

### **Importante:** antes de ejecutar las siguientes celdas, cambien el **correo** por el de ustedes.

In [4]:
Entrez.email = "svillanu@eafit.edu.co"  # Cambienlo por su correo
handle = Entrez.einfo()
result = Entrez.read(handle)
handle.close()

In [9]:
for db in result["DbList"]:
    print(db)

pubmed
protein
nuccore
ipg
nucleotide
nucgss
nucest
structure
sparcle
genome
annotinfo
assembly
bioproject
biosample
blastdbinfo
books
cdd
clinvar
clone
gap
gapplus
grasp
dbvar
gene
gds
geoprofiles
homologene
medgen
mesh
ncbisearch
nlmcatalog
omim
orgtrack
pmc
popset
probe
proteinclusters
pcassay
biosystems
pccompound
pcsubstance
pubmedhealth
seqannot
snp
sra
taxonomy
biocollections
unigene
gencoll
gtr


***
¿Y entonces? A duras pensa reconocemos 5. Pidamos la descripción de cada una a ver si se vuelve más informativo.

In [26]:
print("Base de datos \t Descripción \t número de entradas")
for db in result["DbList"]:
    handle = Entrez.einfo(db=db)
    record = Entrez.read(handle)
    desc = record["DbInfo"]["Description"]
    #print(type(desc))
    entries = record["DbInfo"]["Count"]
    #print(type(entries))
    print("%s \t \t %s \t \t %s" % (db, str(desc), str(entries)))

Base de datos 	 Descripción 	 número de entradas
pubmed 	 	 PubMed bibliographic record 	 	 28882965
protein 	 	 Protein sequence record 	 	 581056830
nuccore 	 	 Core Nucleotide db 	 	 267669951
ipg 	 	 Identical Protein Groups DB 	 	 190454339
nucleotide 	 	 Core Nucleotide db 	 	 267669951
nucgss 	 	 GSS db 	 	 40713027
nucest 	 	 EST db 	 	 77036622
structure 	 	 Three-dimensional molecular model 	 	 143638
sparcle 	 	 Protein classification by domain architecture 	 	 134334
genome 	 	 Genomic sequences, contigs, and maps 	 	 49824
annotinfo 	 	 Annotinfo Database 	 	 834
assembly 	 	 Genome Assembly Database 	 	 212701
bioproject 	 	 BioProject Database 	 	 312433
biosample 	 	 BioSample Database 	 	 9170040
blastdbinfo 	 	 BlastdbInfo Database 	 	 4491394
books 	 	 Books Database 	 	 754531
cdd 	 	 Conserved Domain Database 	 	 56066
clinvar 	 	 ClinVar Database 	 	 444312
clone 	 	 Clone Database 	 	 38326156
gap 	 	 dbGaP Data 	 	 344732
gapplus 	 	 Internal Genotypes and Pheno

***
Un poco desorganizado, pero bueno. Hay descripciones informatias, y otras que lo dejan a uno en las mismas ¿Quién dijo que el NCBI era perfecto?

Notese el número de entradas asociadas a cada base de datos... 
- PubChem tiene identificado más de 247 millones de sustancias
- Existen más de 267 millones de secuencias de nucleotidos
- Hay más de 581 millones de secuencias de proteinas

Los científicos se pasan...

Otra cosa información importante que nos puede retornar **Einfo** es la lista de filtro que podemos usar qen cada base de datos, con el fin de hacer mejores búsquedas

In [13]:
handle = Entrez.einfo(db="pubmed")
rec = Entrez.read(handle)
rec["DbInfo"]["Description"]

'PubMed bibliographic record'

In [31]:
for field in record["DbInfo"]["FieldList"]:
     print("%(Name)s, %(FullName)s, %(Description)s" % field)

ALL, All Fields, All terms from all searchable fields
UID, UID, Unique number assigned to publication
FILT, Filter, Limits the records
WORD, Text Word, Free text associated with record
WRD1, Text Word 1, Free text associated with record with higher score then regular TEXT field
PROP, Properties, Properties of this record
ORG, Organization names, Lab or Clinic name including institution and department
OID, Organization UID, Unique number for this lab/clinic
CITY, Organization City, Lab or clinic city
STATE, Organization State, Lab or clinic State or province
COUNTRY, Organization Country, Lab or clinic Country
POSTCODE, Organization postcode, Lab or clinic zip or postcode
DIRECTOR, Organization Director(s), Lab or clinic director(s)
STAFF, Organization Staff, Lab or clinic Staff name
LS, Lab Service name, Lab Service name
GTRACC, Accession for GTR test, Accession for GTR test
MDAT, Modification Date, The last date on which the record was updated
NAME, name for this test, name for this t

***
Evidentemente, hay decenas de filtros que ~~no~~ utilizar


### **ESearch: Busquedas en las bases de datos**

Conociendo la base de datos a la que queremos acceder,podemoa hacer búsquedas dentro de esta. Por ejemplo, busquemos artículos de bioinformática de colombia en pubmed.

In [32]:
handle = Entrez.esearch(db="pubmed", term="Bioinformatic[ALL] AND Colombia[COUNTRY]")
record = Entrez.read(handle)

Y el resultado es un diccionario que describe los resultados de nuestra búsqueda

In [34]:
record.keys()

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'TranslationStack', 'QueryTranslation'])

In [35]:
record["Count"]

'9'

Por ejemplo, sólo encontró 9 artículos con esa descripción **\#TercerMundo**

La lista de los artículos es la siguiente:

In [38]:
record["IdList"]

['27622633', '26535546', '25504244', '23715190', '24892459', '22030796', '20890564', '19462562', '15962905']

¿Incomprensible verdad? Son Id's asociados a los artículos, nos serán útiles más adelante cuando queramos acceder a ellos.
***
Veamos otro ejemplo, busquemos ahora una secuencia en la base de datos del Genbank. Busquemos una secuencia de interés en el Genbank

In [76]:
handle = Entrez.esearch(db="nucleotide", term="Plukenetia[Orgn] AND matK[Gene]", idtype="acc")
record = Entrez.read(handle)
record["Count"]

'9'

In [79]:
id_list = record["IdList"]

In [80]:
id_list

['MF062253.1', 'GQ463881.1', 'LK021490.1', 'LK021489.1', 'LK021488.1', 'LK021487.1', 'JQ587470.1', 'JQ587469.1', 'AB268032.1']

### **Ejercicio**

Busquen en la base de datos de nucleotidos (o proteinas o metabolitos) una secuencia de interés y guarden los Id que resulten en un lista (Los utilizaremos así que que el resultado tenga **al menos 1** entrada).

### **ESummary: resumenes de Ids**

Cuando utilizamos **ESearch**, obtuvimos una lista de Id's, que son identificadores únicos asociados a los resultados, pero no tenemos ni idea de qué resultados son los que encontramos. Para saberlo, podemos utilizar ESummary. 

In [112]:
id_list[0]

'MF062253.1'

In [81]:
handle = Entrez.esummary(db="nucleotide", id=id_list[0])
record = Entrez.read(handle)

In [89]:
record[0].keys()

dict_keys(['Item', 'Id', 'Caption', 'Title', 'Extra', 'Gi', 'CreateDate', 'UpdateDate', 'Flags', 'TaxId', 'Length', 'Status', 'ReplacedBy', 'Comment', 'AccessionVersion'])

Veamos que información nos retorna

In [98]:
for key in record[0].keys():
    print("%s \t\t %s" % (key,record[0][key]))

Item 		 []
Id 		 1253214023
Caption 		 MF062253
Title 		 UNVERIFIED: Plukenetia volubilis plastid sequence
Extra 		 gi|1253214023|gb|MF062253.1|[1253214023]
Gi 		 1253214023
CreateDate 		 2017/10/15
UpdateDate 		 2018/04/04
Flags 		 256
TaxId 		 316893
Length 		 161733
Status 		 live
ReplacedBy 		 
Comment 		   
AccessionVersion 		 MF062253.1


***
Depndiendo de la base de datos unos campos pueden tener información que otros no. En nuestro caso, para tener una idea de las secuencias de nuestra búsqueda, usaremos caption, title y lenght. 

In [99]:
print("TaxID \t Description \t longitud(bp)")
for idt in id_list: 
    handle = Entrez.esummary(db="nucleotide", id=idt)
    record = Entrez.read(handle)
    print("%s \t %s \t %s" % (record[0]["Caption"],record[0]["Title"],record[0]["Length"]))

TaxID 	 Description 	 longitud(bp)
MF062253 	 UNVERIFIED: Plukenetia volubilis plastid sequence 	 161733
GQ463881 	 Plukenetia volubilis tRNA-Lys (trnK) gene, partial sequence; and maturase K (matK) gene, partial cds; plastid 	 447
LK021490 	 Plukenetia volubilis chloroplast matK gene for maturase K and partial trnK gene intron, specimen voucher G. Ibarra & Sinaca 3608 (MEXU), isolate EPH200 	 2506
LK021489 	 Plukenetia penninervia chloroplast matK gene for maturase K and partial trnK gene intron, specimen voucher D. Alvarez 8184 (MEXU), isolate EPH229 	 2486
LK021488 	 Plukenetia penninervia chloroplast matK gene for maturase K and partial trnK gene intron, specimen voucher A. Hanan 1387 et al. (MEXU), isolate EPH234 	 2520
LK021487 	 Plukenetia penninervia chloroplast matK gene for maturase K and partial trnK gene intron, specimen voucher J. Calonico 22458 (MEXU), isolate EPH228 	 2467
JQ587470 	 Plukenetia stipellata voucher BioBot06277 maturase K (matK) gene, partial cds; chloropla

***
No está mal, así podemos ver cuál secuencia es la que nos interesa para finalmente descargarla.

### **Ejercicio**

Utilicé **ESummary** para obtener información acerca de los Id's que obtuvo en el ejercicio anterior. ¿Qué encontró?

### **EFetch: Descarga  de información de Entrez**

EFetch nos permite descargar las entradas de las bases de datos de NCBI. EFecth recibe varios argumentos. pero siempre es necesario especificar el _rettype_ y/o _retmode_ ,los cuales están en función de la base de datos de que queramos acceder. 

- `db` : Base de datos de la cual descargar. 
- `id` : Identificador
- `rettype` : formato en el cual devolver la información (_e.g._ gb, fasta)
- `retmode` : formato de texto (_e.g._ txt, XML, HTML)


Esta [Tabla](https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/?report=objectonly) describe las combinaciones posibles que se pueden emplear.

Podemos utilizar Efecth para descargar una secuencia que nos interese del ejercicio anterior

In [104]:
handle = Entrez.efetch(db="nucleotide", id=id_list[2], rettype="gb", retmode="text")
print(handle.read())

LOCUS       LK021490                2506 bp    DNA     linear   PLN 04-JAN-2016
DEFINITION  Plukenetia volubilis chloroplast matK gene for maturase K and
            partial trnK gene intron, specimen voucher G. Ibarra & Sinaca 3608
            (MEXU), isolate EPH200.
ACCESSION   LK021490
VERSION     LK021490.1
KEYWORDS    .
SOURCE      chloroplast Plukenetia volubilis
  ORGANISM  Plukenetia volubilis
            Eukaryota; Viridiplantae; Streptophyta; Embryophyta; Tracheophyta;
            Spermatophyta; Magnoliophyta; eudicotyledons; Gunneridae;
            Pentapetalae; rosids; fabids; Malpighiales; Euphorbiaceae;
            Acalyphoideae; Plukenetieae; Plukenetia.
REFERENCE   1
  AUTHORS   Cervantes,A., Magallon,S., Gutierrez-Amaro,J., Fuentes-Bazan,S. and
            Borsch,T.
  TITLE     Phylogeny and Biogeography of Acalyphoideae s. str.
            (Euphorbiaceae): molecular evidence of at least three Caribbean
            lineages with radiations in Cuba and the Hispanola
  J

***
¡Y así obtuvimos un Genbank! Sin embargo, eso así en texto no deja explorar la información facilmente. Por suerte tenemos **SeqIO**

In [105]:
from Bio import SeqIO

In [106]:
handle = Entrez.efetch(db="nucleotide", id=id_list[2], rettype="gb", retmode="text")
my_gb = SeqIO.read(handle, "genbank")

In [111]:
my_gb.description

'Plukenetia volubilis chloroplast matK gene for maturase K and partial trnK gene intron, specimen voucher G. Ibarra & Sinaca 3608 (MEXU), isolate EPH200'

In [109]:
my_gb.seq

Seq('ATCTTTTACACATTTGTATGAAGAAAGGACTTCGTCCATACCATCGGTATAGTT...GGA', IUPACAmbiguousDNA())

### **Ejercicio**

Utilizando **EFetch** descarguen las secuencias de su lista de Id's y escribanas en mismo archivo (formato de libre elección)